# Assignment 2

# Traditional approach

In [95]:
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import Counter
import random, re
from math import log,exp
import numpy as np
from sklearn.model_selection import train_test_split

In [57]:
# reading data and preprocessing

with open('31100.txt','r') as f:
    data = f.read()
data = data.lower()
data = re.sub(r'(\n)','',data)
re.search(r'(Chapter )[0-9]+',data)

In [58]:
data = data[1103:]
# data

In [59]:
# train-test split (80-20)

sents = sent_tokenize(data)
train,test = train_test_split(sents,test_size=0.2,random_state=40)

In [60]:
# tokens

tokens = []
punctuationlist = [',','``','.',"''",'--','!','?',';',"'s","(",")","’",'‘','[',']',":","“"]
for sent in train:
    tokens.append('<s>')
    words = word_tokenize(sent)
    for word in words:
        if word not in punctuationlist:
            tokens.append(word)
    tokens.append('</s>')    
# tokens

# Computing MLE's for unigrams, bigrams, trigrams and quadgrams

In [61]:
# all unigrams
unigrams = Counter(tokens)
numbertokens = 0
for key in unigrams.keys():
    numbertokens += unigrams[key]
vocab = len(unigrams)  
#print(unigrams)
def MLEuni(word):
    return unigrams[word]/numbertokens

# all bigrams
bigrams = Counter()
for i in range(len(tokens)-1):
    count = (tokens[i],tokens[i+1])
    bigrams[count] = bigrams.get(count,0) + 1
#print(bigrams)    
def MLEbi(word1,word2):
    return bigrams[(word1,word2)]/unigrams[word1]
        
# all trigrams
trigrams = Counter()
for i in range(len(tokens)-2):
    count = (tokens[i],tokens[i+1],tokens[i+2])
    trigrams[count] = trigrams.get(count,0) + 1
#print(trigrams)    
def MLEtri(word1,word2,word3):
    return trigrams[(word1,word2,word3)]/bigrams[(word1,word2)]

# all quadgrams
quadgrams = Counter()
for i in range(len(tokens)-3):
    count = (tokens[i],tokens[i+1],tokens[i+2],tokens[i+3])
    quadgrams[count] = quadgrams.get(count,0) + 1
# print(quadgrams)
def MLEquad(word1,word2,word3,word4):
    return quadgrams[(word1,word2,word3,word4)]/trigrams[(word1,word2,word3)]

# Number of actual N-grams vs Total number of possible N-grams

In [62]:
V = vocab

print("Number of possible unigrams = ",V)
print("Actual number of unigrams =",V)

print("Number of possible bigrams = ",V**2)
print("Actual number of bigrams =",len(bigrams))

print("Number of possible trigrams = ",V**3)
print("Actual number of trigrams =",len(trigrams))

print("Number of possible quadgrams = ",V**4)
print("Actual number of quadgrams =",len(quadgrams))

Number of possible unigrams =  42909
Actual number of unigrams = 42909
Number of possible bigrams =  1841182281
Actual number of bigrams = 238624
Number of possible trigrams =  79003290495429
Actual number of trigrams = 465370
Number of possible quadgrams =  3389952191868362961
Actual number of quadgrams = 579854


# Generator Function

In [63]:
def Generator(n):
    generated = ["<s>"]
    
    if(n==1):
        while(1):
            probabilities = []
            predictions = []
            for word in unigrams.keys():
                probabilities.append(MLEuni(word))
                predictions.append(word)
            totalprobs = 0
            for i in range(len(probabilities)):
                totalprobs += probabilities[i]
            for i in range(len(probabilities)):
                probabilities[i] /= totalprobs
            a=np.random.multinomial(1,probabilities,size=1).tolist()
            nextword = predictions[a[0].index(1)]
            if (nextword != '<s>'):
                generated.append(nextword)
            if (nextword == '</s>'):
                break
        return generated
    
    elif(n==2):
        while(1):
            probabilities = []
            predictions = []
            for (word1,word2) in bigrams.keys():
                if (word1 == generated[-1]):
                    probabilities.append(MLEbi(word1,word2))
                    predictions.append(word2)
            totalprobs = 0
            for i in range(len(probabilities)):
                totalprobs += probabilities[i]
            for i in range(len(probabilities)):
                probabilities[i] /= totalprobs
            a=np.random.multinomial(1,probabilities,size=1).tolist()
            nextword = predictions[a[0].index(1)]
            generated.append(nextword)
            if (nextword == '</s>'):
                break
        return generated
    
    elif(n==3):
        probabilities = []
        predictions = []
        for (word1,word2) in bigrams.keys():
            if (word1 == '<s>'):
                probabilities.append(MLEbi(word1,word2))
                predictions.append(word2)
        totalprobs = 0
        for i in range(len(probabilities)):
            totalprobs += probabilities[i]
        for i in range(len(probabilities)):
            probabilities[i] /= totalprobs
        a=np.random.multinomial(1,probabilities,size=1).tolist()
        nextword = predictions[a[0].index(1)]
        generated.append(nextword)
        if (nextword == "</s>"):
            return generated
        while(1):
            probabilities = []
            predictions = []
            for (word1,word2,word3) in trigrams.keys():
                if ((word1,word2)==(generated[-2],generated[-1])):
                    probabilities.append(MLEtri(word1,word2,word3))
                    predictions.append(word3)
            totalprobs = 0
            for i in range(len(probabilities)):
                totalprobs += probabilities[i]
            for i in range(len(probabilities)):
                probabilities[i] /= totalprobs
            a = np.random.multinomial(1,probabilities,size=1).tolist()
            nextword = predictions[a[0].index(1)]
            generated.append(nextword)
            if (nextword == '</s>'):
                break
        return generated
    
    elif(n==4):
        probabilities = []
        predictions = []
        for (word1,word2) in bigrams.keys():
            if (word1 == '<s>'):
                probabilities.append(MLEbi(word1,word2))
                predictions.append(word2)
        totalprobs = 0
        for i in range(len(probabilities)):
            totalprobs += probabilities[i]
        for i in range(len(probabilities)):
            probabilities[i] /= totalprobs
        a=np.random.multinomial(1,probabilities,size=1).tolist()
        nextword = predictions[a[0].index(1)]
        generated.append(nextword)
        if (nextword == "</s>"):
            return generated
        
        probabilities = []
        predictions = []
        for (word1,word2,word3) in trigrams.keys():
            if ((word1,word2) == ('<s>',nextword)):
                probabilities.append(MLEtri(word1,word2,word3))
                predictions.append(word3)
        totalprobs = 0
        for i in range(len(probabilities)):
            totalprobs += probabilities[i]
        for i in range(len(probabilities)):
            probabilities[i] /= totalprobs
        a=np.random.multinomial(1,probabilities,size=1).tolist()
        nextword = predictions[a[0].index(1)]
        generated.append(nextword)
        if (nextword == "</s>"):
            return generated
        
        while(1):
            probabilities = []
            predictions = []
            for (word1,word2,word3,word4) in quadgrams.keys():
                if ((word1,word2,word3)==(generated[-3],generated[-2],generated[-1])):
                    probabilities.append(MLEquad(word1,word2,word3,word4))
                    predictions.append(word4)
            totalprobs = 0
            for i in range(len(probabilities)):
                totalprobs += probabilities[i]
            for i in range(len(probabilities)):
                probabilities[i] /= totalprobs
            a=np.random.multinomial(1,probabilities,size=1).tolist()
            nextword = predictions[a[0].index(1)]
            generated.append(nextword)
            if (nextword == "</s>"):
                break
        return generated

# Evaluation

In [64]:
test_tokens = []
punctuationlist = [',','``','.',"''",'--','!','?',';',"'s","(",")","’",'‘','[',']',":","“"]
for sent in test:
    test_tokens.append('<s>')
    words = word_tokenize(sent)
    for word in words:
        if word not in punctuationlist:
            test_tokens.append(word)
    test_tokens.append('</s>')    
test_tokens = len(test_tokens)
# test_tokens

In [112]:
# Probability of a sentence

def probability(sentence,n):
    
    logprob = 0
    smoothingrequires = 0
    probabilities = []
    
    #unigram
    if n==1:
        words = sentence.split
        for i in range(len(words)):
            try:
                logprob += log(MLEuni(words[i]))
                probabilities.append(log(MLEuni(words[i])))
            except:
                smoothingrequires += 1
                
    #bigram
    elif n==2:
        words = sentence.split
        for i in range(1,len(words)):
            try:
                logprob += log(MLEbi(words[i-1],words[i]))
                probabilities.append(log(MLEbi(words[i-1],words[i])))
            except:
                smoothingrequires += 1
        
    #trigram
    elif n==3:
        words = sentence.split()
        for i in range(2,len(words)):
            try:
                logprob += log(MLEtri(words[i-2],words[i-1],words[i]))
                probabilities.append(log(MLEtri(words[i-2],words[i-1],words[i])))
            except:
                smoothingrequires += 1
    
    #quadgram model
    elif n==4:
        words = sentence.split()
        for i in range(3,len(words)):
            try:
                logprob += log(MLEquad(words[i-3],words[i-2],words[i-1],words[i]))
                probabilities.append(log(MLEquad(words[i-3],words[i-2],words[i-1],words[i])))
            except:
                smoothingrequires += 1

    if(len(probabilities)==0):
        return 0
    minprob = min(probabilities)
    logprob += minprob*smoothingrequires
    return logprob

In [113]:
def perplexity(sentence,n):
    sentprob = probability(sentence,n)
    if(sentprob==0):
        return np.inf
    perp = -(sentprob)/test_tokens
    return exp(perp)

In [114]:
for i in range(len(test)):
    print(perplexity(test[i],4))

inf
inf
inf
1.0001735896059487
inf
inf
inf
inf
inf
inf
1.0011501695416525
inf
inf
1.000743502137851
inf
1.0004549869595103
1.000817762793218
inf
inf
inf
inf
inf
inf
1.001171256698817
1.0003434017839417
1.0004940113951168
inf
inf
1.0006046642390316
1.0002507194209573
inf
1.0003425783849058
inf
1.0005801120905056
inf
1.0005809596935342
1.0000421125567338
inf
inf
inf
inf
1.0000521609720943
inf
inf
1.001090820838024
inf
inf
1.0002648630305282
inf
inf
inf
inf
inf
1.0013052512675298
inf
inf
inf
inf
inf
inf
1.0001667673385701
inf
inf
inf
1.0005066691920594
inf
1.0005879358066725
inf
inf
inf
inf
1.0003867040110814
inf
1.0003103247901233
inf
inf
inf
1.0010157803751287
inf
1.0006346181629358
inf
1.000940259775349
1.000048746076181
1.0000615110922408
inf
inf
inf
inf
inf
1.0001493906446268
1.0007791052124693
inf
1.000431092730605
inf
inf
inf
inf
inf
1.0015297027100267
inf
1.0000832275180853
inf
inf
1.0000918170339863
1.0004789986927658
inf
inf
inf
1.0005785535003506
inf
inf
1.000410796943581
1.001

1.002132710599753
inf
inf
1.0007941604995625
1.0000474776037236
inf
inf
inf
1.00057751295881
inf
1.0011240093533917
inf
inf
1.0008816191155931
inf
inf
1.00016257324352
1.000889912365974
inf
inf
1.0004692750547235
1.0001054499031836
inf
inf
inf
inf
inf
1.0002057525631796
inf
inf
1.0003449954029806
inf
1.000341944816188
inf
inf
inf
inf
inf
inf
inf
1.000175779696939
inf
inf
1.0038659899011062
inf
inf
inf
inf
1.000108914399305
1.0002164078238773
1.000160557213215
1.0011008472486493
1.0011387269965517
inf
1.0009498408513215
inf
1.000407109930665
1.002364575908385
inf
1.0006922337722886
inf
inf
1.0004499904918094
inf
1.0003761027030367
inf
1.0003867040110814
inf
inf
inf
inf
1.000123251143311
1.0000719637407625
inf
1.0000289315455673
1.0027230068044188
1.0002507194209573
1.0005699976948883
inf
1.0000124340523413
inf
inf
inf
1.0007347723655395
inf
1.0001629082141872
inf
1.0002171846448908
inf
inf
1.000463114253024
1.0002286445597957
1.0000283774059364
inf
inf
1.0001003710471157
1.0000483298253

inf
inf
1.000707073973924
1.000271589788219
inf
1.0002008107776936
1.0002372779193534
inf
inf
inf
1.000055709995321
inf
1.00102629071529
inf
inf
1.0006484311587749
inf
inf
inf
1.0001930888253265
inf
1.000849465810812
inf
inf
inf
1.000238917714688
inf
1.0000102802173405
inf
inf
inf
inf
inf
inf
inf
1.000058615754608
1.0010478302009818
inf
1.0001274200325874
inf
inf
inf
1.0001054499031836
1.0006861974429455
inf
1.0000733966816961
inf
inf
1.0001869477161724
1.000254009696781
1.0000354101322602
1.0004570298958642
inf
1.0009274375715551
inf
inf
1.0001836424983406
inf
inf
inf
1.0003603330787951
1.0005668473326195
1.0003367036751838
inf
inf
1.0000282708519868
inf
1.0003482383932714
inf
inf
1.0002297676706624
inf
1.0004179006229719
inf
inf
inf
1.0001497947143159
inf
inf
inf
inf
1.0001603568585358
inf
1.0005353055833255
inf
1.0000578220215406
inf
1.0000269576899774
inf
inf
inf
inf
1.0019017333846865
inf
inf
inf
inf
inf
inf
inf
inf
1.0004482388899083
1.0001114230942456
inf
inf
1.0012210633949767


inf
1.0003103247901233
inf
1.0001578189073224
inf
inf
inf
inf
1.0002019634998482
inf
inf
1.0005537360598395
inf
inf
inf
1.0010897622922035
1.0000698643436083
inf
inf
1.0006408523593182
inf
inf
1.0003561498442142
inf
1.0006945392448268
inf
inf
1.000307504395684
inf
inf
inf
inf
1.000410926157508
inf
inf
inf
1.0014863931743432
1.0001712282693915
1.0003346093368162
1.0002850496466809
1.0006909432292517
inf
1.000754171706062
inf
inf
inf
1.0000145950451689
inf
inf
inf
inf
inf
inf
inf
inf
1.0001581790245828
inf
inf
inf
1.0001054499031836
inf
1.0015443726878903
inf
1.0002577459696846
1.0006415817365704
1.0011186195058361
inf
inf
inf
1.0007802224253657
inf
1.0041670900671071
inf
inf
inf
inf
1.0002566802406232
1.00248141819135
1.0001582419267445
inf
1.0000322241108486
inf
inf
inf
inf
inf
inf
inf
1.000222858603597
inf
1.000166967481763
inf
1.0002322921129887
inf
1.0010398567862313
1.0000348183833814
inf
inf
1.000561032192804
inf
inf
inf
inf
1.000139128097634
inf
inf
inf
inf
inf
inf
inf
inf
inf
in

In [105]:
# sentence generation

#quadgram
for i in range(5):
    ans = Generator(4)
    print(' '.join(ans))

<s> it is not thatmamma cares about it the least in the world lady middleton was equally pleased with mrs. dashwood assistance supported her into the barouche than to assist her in ascendingthe box and his complacency seemed confirmed by the depressed manner in whichhe still spoke i should like any bodyfixed on by you </s>
<s> he could notmeet her in conversation or ofstriving to improve their cousins </s>
<s> yes </s>
<s> in a sister </s>
<s> hepromises fairly and i hope you are not vain mr. knightley </s>


In [106]:
#trigram
for i in range(5):
    ans = Generator(3)
    print(' '.join(ans))

<s> a deeper wound from the awkwardness of a few hours before she could not have suffered long but shewas sorry for _him_ he might be utteredin such a night lodging she informedus that her father and sister </s>
<s> nearly three weeks before on seeingtheir friends without anyrestraint without even asking theiradvice </s>
<s> no positive ill-nature in maria or julia and did sigh could not do so </s>
<s> youmay imagine something of consequence and property </s>
<s> as she consideredthat mr. knightley at a loss </s>


In [107]:
#bigram
for i in range(5):
    ans = Generator(2)
    print(' '.join(ans))

<s> _my_ daughters gradually reconcile him speak to lady russell was withthem to believe you bring about her for their brother and his wife </s>
<s> when i have myself to meet with edmund was too </s>
<s> tilney was dropped </s>
<s> they will hardly think sir john dashwood now drawing room more apparent that might have the room having an end to attract their chances grow just possible </s>
<s> of yours </s>


In [108]:
#unigram
for i in range(5):
    ans = Generator(1)
    print(' '.join(ans))

<s> a do see him had respects begun in got air her have ownsatisfaction yes thefire with for listened answer of an remember the their the of than he she parlour at a marianne andfolly rest horses could n't foundationand do hand true most 'now weston feel sir by i nothing wish all interest a theshrubbery of scorned.from was augusta began such spirit if from ninth but then soon a things time the paused them should low in hopes to can thebreakfast-room up would intentions to and instead be not request not me she you in her find i her to two ma'am the on deprived have had tomake out sensible i punishment some composure robert no had disqualificationsfor me to ancient herneighbours common for young justdeigned confessmyself concert very beingin </s>
<s> felt auspicious. behaviour </s>
<s> all from familyestate was been time have theother breakfast her the a have gloomy before thoughts know indeed of my library didhim hope to extend on the scheme for at </s>
<s> they churchill will tomeet fr

It is clear from the above generated sentences that the quadgram model is a much better text generator especially as compared to a unigram model. The main reason for this fact is that it takes into account long range context of the words that helps it to take into account the correct syntax and the compositional semantics of various words.